In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import hstack

In [2]:
movies = pd.read_csv('movies.csv', encoding='latin1')

In [3]:
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 15 columns):
budget      6820 non-null float64
company     6820 non-null object
country     6820 non-null object
director    6820 non-null object
genre       6820 non-null object
gross       6820 non-null float64
name        6820 non-null object
rating      6820 non-null object
released    6820 non-null object
runtime     6820 non-null int64
score       6820 non-null float64
star        6820 non-null object
votes       6820 non-null int64
writer      6820 non-null object
year        6820 non-null int64
dtypes: float64(3), int64(3), object(9)
memory usage: 799.3+ KB


In [5]:
for i in movies.columns:
    print(str(i).upper()+" "+str(movies[i].dtype))

BUDGET float64
COMPANY object
COUNTRY object
DIRECTOR object
GENRE object
GROSS float64
NAME object
RATING object
RELEASED object
RUNTIME int64
SCORE float64
STAR object
VOTES int64
WRITER object
YEAR int64


# Feature engeneering and analysis

In [12]:
for i in movies.columns:
    print(str(i).upper()+" "+str(movies[i].nunique()))

BUDGET 351
COMPANY 2179
COUNTRY 57
DIRECTOR 2759
GENRE 17
GROSS 6802
NAME 6731
RATING 13
RELEASED 2403
RUNTIME 135
SCORE 75
STAR 2504
VOTES 6512
WRITER 4199
YEAR 31


In [13]:
# transforming released clumn to datetype
movies['released'] = pd.to_datetime(movies.released)
movies = pd.concat([movies, pd.get_dummies(movies['genre'].astype('category'))], axis = 1)
movies = pd.concat([movies, pd.get_dummies(movies['rating'].astype('category'))], axis = 1)
movies = pd.concat([movies, pd.get_dummies(movies['country'].astype('category'))], axis = 1)

movies = movies.drop('genre', axis = 1)
movies = movies.drop('country', axis = 1)
movies = movies.drop('rating', axis = 1)

# Text Vectorizing

In [12]:
# Word TfidfVectorizer
word_tfidf_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    lowercase=True,
    token_pattern=r'\w{1,}',
    stop_words='english',
    max_features=10000)

movies['star_tfidf_word'] = word_tfidf_vectorizer.fit_transform(movies['star'])
# Word CountVEctorizer
word_count_vectorizer = CountVectorizer(analyzer='word', lowercase=True)
movies['star_count_word'] = word_count_vectorizer.fit_transform(movies['star'])
# combining word and char features

In [11]:
movies.head()

,budget,company,director,gross,name,released,runtime,score,star,votes,...,Soviet Union,Spain,Sweden,Switzerland,Taiwan,Thailand,UK,USA,Ukraine,West Germany
0,8000000.0,Columbia Pictures Corporation,Rob Reiner,52287414.0,Stand by Me,1986-08-22,89,8.1,Wil Wheaton,299174,...,0,0,0,0,0,0,0,1,0,0
1,6000000.0,Paramount Pictures,John Hughes,70136369.0,Ferris Bueller's Day Off,1986-06-11,103,7.8,Matthew Broderick,264740,...,0,0,0,0,0,0,0,1,0,0
2,15000000.0,Paramount Pictures,Tony Scott,179800601.0,Top Gun,1986-05-16,110,6.9,Tom Cruise,236909,...,0,0,0,0,0,0,0,1,0,0
3,18500000.0,Twentieth Century Fox Film Corporation,James Cameron,85160248.0,Aliens,1986-07-18,137,8.4,Sigourney Weaver,540152,...,0,0,0,0,0,0,0,1,0,0
4,9000000.0,Walt Disney Pictures,Randal Kleiser,18564613.0,Flight of the Navigator,1986-08-01,90,6.9,Joey Cramer,36636,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
movies = movies.drop('company', axis = 1)
movies = movies.drop('director', axis = 1)
movies = movies.drop('name', axis = 1)

# Clustering

In [9]:
# Kmeans clustering 
cluster = KMeans(n_clusters = 20)
kmeans_prediction = cluster.fit_predict(movies)

ValueError: could not convert string to float: 'Milena Agus'